In [1]:
%env HF_ENDPOINT=https://hf-mirror.com
from KeepFIT.keepfit.modeling.model import KeepFITModel
from retriever import TextRetriever, MMRetriever
import torch


env: HF_ENDPOINT=https://hf-mirror.com


/home/jqxu/anaconda3/envs/mmretinal/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mmRetriever=MMRetriever(device=device,dataset_path="/home/jqxu/Ragas/MM_feature_set")
textRetriever=TextRetriever(device=device,dataset_path="/home/jqxu/Ragas/text_datasets/dpr-txt/combined_books")
model = KeepFITModel(vision_type='resnet_v2', out_path='./output', from_checkpoint=True,
                        vision_pretrained=True,
                        weights_path=f'/home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth')
model.to(device)

Pretrained weights: IMAGENET1K_V2


/home/jqxu/anaconda3/envs/mmretinal/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


load model weight from: /home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth


100%|██████████| 6/6 [00:00<00:00, 356.15it/s]


Pretrained weights: IMAGENET1K_V2
load model weight from: /home/jqxu/Ragas/KeepFIT/results/pretraining/50%flair+MM/KeepFIT (50%flair+MM).pth


KeepFITModel(
  (vision_model): VisionModel(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): R

In [3]:
from extract.extractor import Extractor
from query_processor import QueryProcessor
import datasets
import numpy as np
import random
from PIL import Image
api_key = "REMOVED"
mmdataset = datasets.load_from_disk("/home/jqxu/Ragas/MM_feature_set")

keyWordExtractor=Extractor(model='gpt-3.5-turbo',api_key=api_key)
headers={
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
queryProcessor=QueryProcessor(headers)

In [4]:
import os
'''
random_indices=random.sample(range(len(mmdataset)), 20)
mmSamples = mmdataset.select(indices=random_indices)
'''
for sample in mmdataset:
    img_path = sample['image_path']
    img_name=os.path.basename(img_path)
    check_path = os.path.join("/home/jqxu/Ragas/datasets",img_name)
    if not os.path.exists(check_path):
        print(f"Already exists {img_name}")
        continue
    '''
    scores, retrieved_examples = mmRetriever.retrieve(img_path, k=1)
    similar_img_path = retrieved_examples["image_path"][0]
    similar_description = retrieved_examples["description"][0]
    print("Image Description:",sample["description"])
    print("similar Description:",similar_description)
    keywords=keyWordExtractor(similar_description)
    print(f"extracted keywords: {keywords}")
    scores,retrieved_texts = textRetriever.retrieve(query=keywords, k=3,is_keywords=True)
    combined_texts = [f"{title}: {text}" for title, text in zip(retrieved_texts["title"],retrieved_texts["text"])]
    '''
    result=queryProcessor.process_query(image_path=img_path)
    
    #save the text to .txt file
    with open("result_withoutRag.txt", "a", encoding="utf-8") as f:
        f.write(f"Image path: {img_path}\n")
        for choice in result['choices']:
            message_content = choice['message']['content']
            f.write(message_content + "\n")
            f.write("-" * 100 + "\n") 
        print("-" * 100)


Already exists figure01-01A.jpg
Already exists figure01-01B.jpg
Already exists figure01-02A.jpg
Already exists figure01-02B.jpg
Already exists figure01-03A.jpg
Already exists figure01-03B.jpg
Already exists figure01-05A.jpg
Already exists figure02-01A.jpg
Already exists figure02-01B.jpg
Already exists figure02-02A.jpg
Already exists figure02-02B.jpg
Already exists figure02-03A.jpg
Already exists figure02-04A.jpg
Already exists figure02-05A.jpg
Already exists figure02-06A.jpg
Already exists figure02-07A.jpg
Already exists figure02-08A.jpg
Already exists figure02-09A.jpg
Already exists figure02-09B.jpg
Already exists figure02-10A.jpg
Already exists figure02-10B.jpg
Already exists figure02-11A.jpg
Already exists figure02-12.jpg
Already exists figure02-13.jpg
Already exists figure02-14A.jpg
Already exists figure02-15A.jpg
Already exists figure02-15B.jpg
Already exists figure02-16.jpg
Already exists figure02-17.jpg
Already exists figure02-18.jpg
Already exists figure02-19A.jpg
Already exist